In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
import re
import numpy as np
from datetime import datetime

file_paths = glob.glob('data/*.html')
list_of_dfs = []
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    header = soup.find('div', class_='particle-table-header')

    if not header:
        print(f"  --> Warning: Could not find a header in {file_path}. Skipping this file.")
        continue

    columns = [cell.text.strip() for cell in header.find_all('aw-header-cell')]

    rows = []
    for row in soup.find_all('div', class_='particle-table-row'):
        cells = row.find_all('ess-cell')
        if len(cells) == len(columns):
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
        else:
            print(f"  --> Warning: Mismatch between column count and cell count in a row within {file_path}.")

    df_current_file = pd.DataFrame(rows, columns=columns)

    date_str_match = re.search(r'keywords_(\d{2}-\d{2}-\d{2})\.html', file_path)
    if date_str_match:
        date_str = date_str_match.group(1)
        year_part = date_str.split('-')[0]
        full_year_str = '20' + year_part
        corrected_date_str = f"{full_year_str}-{date_str.split('-')[1]}-{date_str.split('-')[2]}"
        df_current_file['date'] = pd.to_datetime(corrected_date_str)
    else:
        df_current_file['date'] = pd.NaT


    list_of_dfs.append(df_current_file)

if list_of_dfs:
    df = pd.concat(list_of_dfs, ignore_index=True)
else:
    print("No valid HTML files were found or processed. The final DataFrame is empty.")
    df = pd.DataFrame()

def to_snake_case(column_name):
    s = re.sub(r'[\s\n]+', ' ', column_name).strip()
    s = s.lower()
    s = re.sub(r'[().]', '', s)
    s = s.replace(' ', '_')
    s = s.replace('top_of_page_', '')
    return s

original_columns = df.columns
new_columns_map = {col: to_snake_case(col) for col in original_columns}
df.rename(columns=new_columns_map, inplace=True)

df = df.drop(columns=['account_status', 'ad_impression_share'])

df['competition'] = df['competition'].map({'Low': 1, 'Medium': 2, 'High': 3})
df['keyword'] = df['keyword'].apply(lambda x: re.sub(r'[\s\n]+', ' ', x).strip())

df['avg_monthly_searches'] = df['avg_monthly_searches'].str.split(' –').str[0]
df['avg_monthly_searches'] = df['avg_monthly_searches'].str.replace('K', '000', regex=False)
df['avg_monthly_searches'] = df['avg_monthly_searches'].str.replace('M', '000000', regex=False)
df['avg_monthly_searches'] = pd.to_numeric(df['avg_monthly_searches'], errors='coerce')

for col in ['bid_low_range', 'bid_high_range']:
    df[col] = df[col].str.replace('€', '', regex=False)
    df[col] = df[col].str.replace('—', '', regex=False).str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['avg_bid'] = df[['bid_high_range', 'bid_low_range']].mean(axis=1)

for col in ['three_month_change', 'yoy_change']:
    df[col] = df[col].str.replace('%', '', regex=False)
    df[col] = df[col].str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] /= 100

df = (df.sort_values(by='date')
        .drop_duplicates(subset='keyword', keep='last'))

df['keyword_len'] = df['keyword'].apply(lambda x: len(x))
df['keyword_len'] = (df['keyword_len'] - df['keyword_len'].min()) / (df['keyword_len'].max() - df['keyword_len'].min())

df['avg_monthly_searches_norm'] = np.log1p(df['avg_monthly_searches'])
df['avg_monthly_searches_norm'] = (df['avg_monthly_searches_norm'] - df['avg_monthly_searches_norm'].min()) / (df['avg_monthly_searches_norm'].max() - df['avg_monthly_searches_norm'].min())

df['score'] = df['avg_monthly_searches_norm'] - df['keyword_len']

df = df.sort_values(by='score', ascending=False)
df

In [ ]:
import json
import numpy as np
from datetime import datetime, timezone
from time import sleep
from urllib.parse import quote, urlparse, parse_qs

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from spacy.lang.en.stop_words import STOP_WORDS


def setup_driver(url="https://trends.google.com/trends/explore"):
    options = webdriver.ChromeOptions()
    options.set_capability("goog:loggingPrefs", {"performance": "INFO"})
    options.add_argument("--no-sandbox")
    # options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd("Network.enable", {})
    driver.get(url)
    return driver


def parse_json_body(entry):
    body = entry.get("body", "{}")
    if body.startswith(")]}'"):
        body = body.split("\n", 1)[1]
    return json.loads(body)


def get_multiline_data(driver, log_json):
    request_id = log_json["params"]["requestId"]
    response_body = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    parsed = parse_json_body(response_body)

    timeline = parsed["default"]["timelineData"]
    dates = np.array([
        datetime.fromtimestamp(int(t["time"]), tz=timezone.utc).date()
        for t in timeline
    ])

    values_list = [t.get("value", []) for t in timeline]
    values = np.array(values_list, dtype=float)
    if values.ndim == 1:
        values = values.reshape(-1, 1) # Ensure values is 2D

    maximums = np.max(values, axis=0)

    return maximums, dates, values


def get_related_keywords(driver, log_json):
    full_url = log_json["params"]["response"]["url"]
    query_params = parse_qs(urlparse(full_url).query)
    query_req = json.loads(query_params['req'][0])
    keyword = query_req['restriction']['complexKeywordsRestriction']['keyword'][0]['value']

    request_id = log_json["params"]["requestId"]
    response_body = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    parsed = parse_json_body(response_body)
    related_keywords = parsed["default"]["rankedList"]
    
    query_value_dict = {}
    for keyword_dict in related_keywords[0]['rankedKeyword']:
        query_value_dict[keyword_dict['query']] = keyword_dict['value']

    return keyword, query_value_dict


def build_batches(driver, sorted_keywords, wait, refetch_rank=2, max_retries=5):
    batches = []
    i = 0
    prev_shared = None
    retry_count = 0

    while i < len(sorted_keywords):
        if prev_shared:
            batch_keywords = [prev_shared] + sorted_keywords[i:i+4]
            i += 4
        else:
            batch_keywords = sorted_keywords[i:i+5]
            i += 5
        if not batch_keywords: 
            break

        # clear stray logs
        logs = driver.get_log("performance") 
        while logs:
            logs = driver.get_log("performance")

        encoded = [quote(kw) for kw in batch_keywords]
        url = f"https://trends.google.com/trends/explore?q={','.join(encoded)}&hl=en"
        driver.get(url)
        wait.until(EC.presence_of_element_located((By.ID, "TIMESERIES")))
        sleep(1)

        current_batch_data = {
            "keywords": batch_keywords,
            "related_keywords": {}
        }

        logs = driver.get_log("performance")
        all_logs = logs
        while logs:
            sleep(1)
            logs = driver.get_log("performance")
            all_logs.extend(logs)

        try:
            for log in all_logs:
                try:
                    log_json = json.loads(log["message"])["message"]
                    if log_json.get("method") == "Network.responseReceived":
                        response_url = log_json["params"]["response"]["url"]
                        if "api/widgetdata/multiline" in response_url:
                            maximums, dates, values = get_multiline_data(driver, log_json)
                            current_batch_data['dates'] = dates
                            current_batch_data['values'] = values
                        elif "api/widgetdata/relatedsearches" in response_url:
                            keyword, related = get_related_keywords(driver, log_json)
                            current_batch_data['related_keywords'][keyword] = related
                except (json.JSONDecodeError, KeyError):
                    continue
        except Exception:
            print(f"Failed to use request_id. Retrying")
            retry_count += 1
            if retry_count > max_retries:
                raise RuntimeError(f"Exceeded max retries ({max_retries}) for batch {batch_keywords}")
            if prev_shared:
                i -= 4
            else:
                i = 0
            sleep(1)
            continue

        if 'values' in current_batch_data:
            batches.append(current_batch_data)
            retry_count = 0  # reset retry counter on success
        else:
            print(f"Failed to retrieve multiline data. Retrying")
            retry_count += 1
            if retry_count > max_retries:
                raise RuntimeError(f"Exceeded max retries ({max_retries}) for batch {batch_keywords}")
            if prev_shared:
                i -= 4
            else:
                i = 0
            sleep(1)
            continue

        # Pick refetch_rank keyword max for the next iteration
        keyword_means = sorted(
            zip(maximums, batch_keywords), 
            key=lambda x: x[0], 
            reverse=True
        )
        if len(keyword_means) >= refetch_rank:
            prev_shared = keyword_means[refetch_rank - 1][1]
        else:
            prev_shared = keyword_means[-1][1] if keyword_means else None
        if not prev_shared:
            break

    return batches


def stitch_batches(batches):
    if not batches:
        return [], np.array([])

    stitched_keywords = list(batches[0]["keywords"])
    stitched_values = np.array(batches[0]["values"])
    if stitched_values.ndim == 1:
        stitched_values = stitched_values[:, np.newaxis]

    for i in range(1, len(batches)):
        prev = batches[i - 1]
        current = batches[i]

        current_values = np.array(current["values"])
        if current_values.ndim == 1:
            current_values = current_values[:, np.newaxis]

        # find shared keyword
        shared = list(set(prev["keywords"]) & set(current["keywords"]))[0]

        prev_idx = stitched_keywords.index(shared)
        current_idx = current["keywords"].index(shared)

        mean_prev = stitched_values[:, prev_idx].mean()
        mean_current = current_values[:, current_idx].mean()
        scale = 0.0 if mean_current == 0 else mean_prev / mean_current

        scaled = current_values * scale
        keep_cols = [j for j, kw in enumerate(current["keywords"]) if kw not in stitched_keywords]

        stitched_values = np.column_stack((stitched_values, scaled[:, keep_cols]))
        stitched_keywords.extend([current["keywords"][j] for j in keep_cols])

    return stitched_keywords, stitched_values


def get_trends_data(driver, wait, sorted_keywords, refetch_rank=2):
    if not sorted_keywords:
        return [], np.array([]), {}

    driver.get_log("performance")  # clear stray logs
    batches = build_batches(driver, sorted_keywords, wait, refetch_rank=refetch_rank)

    stitched_keywords, stitched_values = stitch_batches(batches)

    related_keywords = {}
    for batch in batches:
        for main_keyword, related_data_dict in batch.get("related_keywords", {}).items():
            if main_keyword not in related_keywords:
                related_keywords[main_keyword] = related_data_dict

    trends_df = pd.DataFrame(stitched_values, columns=stitched_keywords)
    return trends_df, related_keywords, batches


def resort_df(df, trends_df):
    temp = pd.DataFrame(trends_df.mean()).reset_index().rename(columns={'index':'keyword', 0:'scrape_vol'})
    df = df.drop(columns='scrape_vol', errors='ignore')
    df = df.merge(temp, on='keyword')
    return df.sort_values(by=['scrape_vol', 'score'], ascending=[False, False])


def create_unified_keyword_series_refined(trends_df, related_keywords):
    sorted_scalars = trends_df.mean(axis=0)

    related_data = []
    for parent_kw, rel_dict in related_keywords.items():
        if parent_kw not in sorted_scalars:
            continue
        parent_mean = sorted_scalars[parent_kw]
        for rel_kw, score in rel_dict.items():
            scaled_val = (score / 100) * parent_mean
            related_data.append((rel_kw, scaled_val))

    if related_data:
        related_df = pd.DataFrame(related_data, columns=['keyword', 'score'])
        related_scalars = related_df.groupby('keyword')['score'].mean()
    else:
        related_scalars = pd.Series(dtype=float)

    all_scalars = (
        pd.concat([sorted_scalars, related_scalars])
        .groupby(level=0).mean()
        .sort_values(ascending=False)
    )

    return all_scalars


driver = setup_driver()
wait = WebDriverWait(driver, 20)
input("Press Enter after login/CAPTCHA if needed...")

print('Fetch 1...')
sorted_keywords = df.keyword.to_list()
trends_df, related_keywords, batches = get_trends_data(driver, wait, sorted_keywords, refetch_rank=2)
for i in range(2):
    print(f'Fetch {i+2}...')
    df = resort_df(df, trends_df)
    sorted_keywords = df.keyword.to_list()
    trends_df, related_keywords, batches = get_trends_data(driver, wait, sorted_keywords, refetch_rank=3)
driver.quit()

all_keywords = create_unified_keyword_series_refined(trends_df, related_keywords)
all_keywords = all_keywords[all_keywords.index.isin([kw for kw in all_keywords.index if kw not in STOP_WORDS])]
all_keywords = all_keywords.reset_index()
all_keywords.columns = ['keyword', 'estimated_relative_volume']

if input('save? (y/n)').lower().strip() == 'y':
    df = resort_df(df, trends_df)
    df['date'] = df['date'].dt.date
    df.to_csv('data/planner_keywords_sorted.csv', index=False)
    all_keywords.to_csv('data/all_keywords.csv', index=False)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

def create_unified_keyword_series_refined(trends_df, related_keywords):
    sorted_scalars = trends_df.mean(axis=0)

    related_data = []
    for parent_kw, rel_dict in related_keywords.items():
        if parent_kw not in sorted_scalars:
            continue
        parent_mean = sorted_scalars[parent_kw]
        for rel_kw, score in rel_dict.items():
            scaled_val = (score / 100) * parent_mean
            related_data.append((rel_kw, scaled_val))

    if related_data:
        related_df = pd.DataFrame(related_data, columns=['keyword', 'score'])
        related_scalars = related_df.groupby('keyword')['score'].mean()
    else:
        related_scalars = pd.Series(dtype=float)

    all_scalars = (
        pd.concat([sorted_scalars, related_scalars])
        .groupby(level=0).mean()
        .sort_values(ascending=False)
    )

    return all_scalars

all_keywords = create_unified_keyword_series_refined(trends_df, related_keywords)
all_keywords = all_keywords[all_keywords.index.isin([kw for kw in all_keywords.index if kw not in STOP_WORDS])]
all_keywords = all_keywords.reset_index()
all_keywords.columns = ['keyword', 'estimated_relative_volume']
all_keywords.to_csv('data/all_keywords.csv', index=False)

In [ ]:
from gliner import GLiNER

all_keywords[all_keywords.index.str.split().str.len() == 1].index

# model = GLiNER.from_pretrained("urchade/gliner_large-v2.1")
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""

labels = ["company", "brand", "organization", "product"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


In [ ]:
driver = setup_driver()
wait = WebDriverWait(driver, 20)
input("Press Enter after login/CAPTCHA if needed...")

all_scalars_filtered = all_scalars[all_scalars.index.str.split().str.len() > 1]
sorted_keywords = all_keywords.index.to_list()
trends_df, related_keywords, batches = get_trends_data(driver, wait, sorted_keywords, refetch_rank=2)

for _ in range(2):
    sorted_keywords = pd.DataFrame(trends_df.mean()).reset_index().rename(columns={'index':'keyword', 0:'scrape_vol'}).keyword.to_list()
    trends_df, related_keywords, batches = get_trends_data(driver, wait, sorted_keywords, refetch_rank=3)

In [ ]:
all_keywords[all_keywords.index.str.split().str.len() == 1].index

In [ ]:
def build_keyword_scalars(trends_df, related_keywords):
    sorted_scalars = trends_df.mean(axis=0)
    
    related_scalars = []
    for parent_kw, rel_dict in related_keywords.items():
        if parent_kw not in sorted_scalars:
            continue
        parent_mean = sorted_scalars[parent_kw]
        for rel_kw, score in rel_dict.items():
            scaled_val = (score / 100) * parent_mean
            related_scalars.append((rel_kw, scaled_val))
    
    related_scalars = pd.Series(dict(related_scalars))
    
    all_scalars = (
        pd.concat([sorted_scalars, related_scalars])
        .groupby(level=0).mean()
        .sort_values(ascending=False)
    )
    
    return all_scalars
